In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle competitions download -c visual-taxonomy

visual-taxonomy.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# """
import zipfile
with zipfile.ZipFile('visual-taxonomy.zip','r') as zip_ref:
  zip_ref.extractall()
  zip_ref.close()
# """

In [ ]:
import pandas as pd
import gc

In [ ]:
def load_df_train(idx):
  df = pd.read_csv('train.csv')
  df_idx = df[df['Category']==categories[idx]]
  del df
  gc.collect()
  return df_idx

In [ ]:
categories = ['Men Tshirts', 'Sarees' ,'Kurtis' ,'Women Tshirts' ,'Women Tops & Tunics']
print(categories)

['Men Tshirts', 'Sarees', 'Kurtis', 'Women Tshirts', 'Women Tops & Tunics']


In [ ]:
df_idx = load_df_train(1)

In [ ]:
categories_consider = ['attr_2','attr_4','attr_5','attr_7','attr_9']

In [ ]:
df_attrs = []
for i in range(len(categories_consider)):
  df_attrs.append(df_idx[['id',categories_consider[i]]])
# print(df_shirt_attr1.head())

In [ ]:
# print(df_shirt_attr1.isnull().mean())
filtered_df_idx = []
for i,attr in enumerate(categories_consider):
  df_attr = df_attrs[i]
  filtered_df = df_attr[df_attr[attr].notnull()]
  filtered_df_idx.append(filtered_df)

In [ ]:
unique_attr_class = []
unique_attr_class_len = []
for i,attr in enumerate(categories_consider):
  unique_attrs = filtered_df_idx[i][attr].unique().tolist()
  unique_attr_class.append(unique_attrs)
  unique_attr_class_len.append(len(unique_attrs))
  print(f' {attr} unique attributes {unique_attr_class[i]} and lenght is {len(filtered_df_idx[i][attr].unique())}')

 attr_2 unique attributes ['woven design', 'zari', 'no border', 'solid', 'default', 'temple border'] and lenght is 6
 attr_4 unique attributes ['multicolor', 'cream', 'white', 'default', 'navy blue', 'yellow', 'green', 'pink'] and lenght is 8
 attr_5 unique attributes ['party', 'traditional', 'daily', 'wedding'] and lenght is 4
 attr_7 unique attributes ['woven design', 'same as saree', 'default', 'zari woven'] and lenght is 4
 attr_9 unique attributes ['applique', 'elephant', 'floral', 'ethnic motif', 'peacock', 'default', 'solid', 'checked', 'botanical'] and lenght is 9


In [ ]:
import numpy as np

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
y_attrs = []
for i,attr in enumerate(categories_consider):
  encoder = OrdinalEncoder(categories=[unique_attr_class[i]])
  y = filtered_df_idx[i]
  y = encoder.fit_transform(y[[attr]])
  y = np.array(y)
  y_attrs.append(y)

In [ ]:
import tensorflow as tf

In [ ]:
def load_images(paths,n_images = 100, size=(128,128)):
  images = []
  for i,path in enumerate(paths):
    if i == n_images:
      break
    image = tf.keras.preprocessing.image.load_img(path,target_size=size)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image/255.0
    images.append(image)
  return np.array(images)

In [ ]:
import os

In [ ]:
img_paths = os.listdir('train_images')
img_paths = [os.path.join('train_images',path) for path in img_paths]
img_paths.sort()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import sys
def get_size(data):
  size = sys.getsizeof(data)
  print(f'The size in GB is {size/1024**3}')

In [ ]:
id_no_null = []
for i in range(len(categories_consider)):
  no_null = np.array(filtered_df_idx[i]['id'])
  id_no_null.append(no_null)

In [ ]:
filtered_img_paths = []
for j in range(len(categories_consider)):
  filtered_img_path = [img_paths[i] for i in id_no_null[j]]
  filtered_img_paths.append(filtered_img_path)

In [ ]:
from PIL import Image
import numpy as np
import torchvision.transforms as T

In [ ]:
def augment_image(np_image):
    image = Image.fromarray((np_image * 255).astype(np.uint8))
    original_size = image.size

    transform = T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.5),
        T.RandomRotation(degrees=10),
        T.ColorJitter(brightness=0.1,
                      contrast=0.1,
                      saturation=0.05,
                      hue=0.02),
        T.CenterCrop(size=original_size)

    ])
    augmented_image = transform(image)
    augmented_np_image = np.array(augmented_image)
    return augmented_np_image
def generate_aument_images(np_images):
  augmented_images = []
  for i in range(len(np_images)):
    augmented_image = augment_image(np_images[i])
    augmented_image = augmented_image/255.0
    augmented_images.append(augmented_image)
  return np.array(augmented_images)

In [ ]:
# img = load_images(filtered_img_paths[0])
# aug_img = generate_aument_images(img)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam

In [ ]:
def define_model(n_classes,input_shape, each_class_attr):
  input_layer = Input(shape=input_shape)

  x = Conv2D(32, (3, 3), activation='relu')(input_layer)
  x = MaxPooling2D((2, 2))(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(64, (3, 3), activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Flatten()(x)

  x = Dense(128, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = BatchNormalization()(x)


  output_layers = []
  loss_functions = {}
  metrics = {}
  for i in range(n_classes):
    output_name = f'attr{i+1}_output'
    y = Dense(each_class_attr[i], activation='softmax', name=output_name)(x)
    loss_functions[output_name] = 'sparse_categorical_crossentropy'
    metrics[output_name] = 'accuracy'
    output_layers.append(y)
  model = Model(inputs=input_layer, outputs=output_layers)
  opt = Adam(0.0002)
  model.compile(optimizer=opt, loss=loss_functions, metrics=metrics)
  return model



In [ ]:
saree_attr7 = define_model(n_classes=1, input_shape=(128,128,3), each_class_attr=[4])

In [ ]:
saree_attr7.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 63, 63, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 59, 59, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 59, 59, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 59, 59, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 445568)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │      57,032,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attr1_output (Dense)                 │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 57,128,004 (217.93 MB)

 Trainable params: 57,127,300 (217.92 MB)

 Non-trainable params: 704 (2.75 KB)

In [ ]:
def train_model(x_train, y_train,x_val,y_val, model, batch_size=64, epochs=10):
  model.fit(x_train, y_train, validation_data=(x_val,y_val),batch_size=batch_size, epochs=epochs)

In [ ]:
from sklearn.model_selection import train_test_split
def del_and_load_images(idx,x_train,x_val,y_train,y_val):
  del x_train, x_val, y_train, y_val,
  gc.collect()
  y = y_attrs[idx]
  x = load_images(filtered_img_paths[idx], n_images=len(y), size=(128,128))
  x_train, x_val, y_train, y_val = train_test_div(x,y)
  del x,y
  gc.collect()
  return x_train, x_val, y_train, y_val

In [ ]:
from sklearn.model_selection import train_test_split
def train_test_div(x,y):
  x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
  return x_train, x_val, y_train, y_val

In [ ]:
x_train,x_val,y_train,y_val = del_and_load_images(3,0,0,0,0)

In [ ]:
train_model(x_train,y_train,x_val,y_val,saree_attr7,epochs=10)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


112/112 ━━━━━━━━━━━━━━━━━━━━ 37s 191ms/step - accuracy: 0.2414 - loss: 0.0000e+00 - val_accuracy: 0.2185 - val_loss: 0.0000e+00
Epoch 2/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 15s 73ms/step - accuracy: 0.2417 - loss: 0.0000e+00 - val_accuracy: 0.2185 - val_loss: 0.0000e+00
Epoch 3/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.2441 - loss: 0.0000e+00 - val_accuracy: 0.2185 - val_loss: 0.0000e+00
Epoch 4/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.2317 - loss: 0.0000e+00 - val_accuracy: 0.2185 - val_loss: 0.0000e+00
Epoch 5/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.2440 - loss: 0.0000e+00 - val_accuracy: 0.2185 - val_loss: 0.0000e+00
Epoch 6/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - accuracy: 0.2468 - loss: 0.0000e+00 - val_accuracy: 0.2185 - val_loss: 0.0000e+00
Epoch 7/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - accuracy: 0.2383 - loss: 0.0000e+00 - val_accuracy: 0.2185 - val_loss: 0.0000e+00
Epoch 8/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 74

In [ ]:
del x_train,x_val,y_train,y_val, filtered_df, filtered_df_idx, df_idx, df_attrs, filtered_img_paths, img_paths, unique_attr_class, unique_attr_class_len, id_no_null, y_attrs
gc.collect()

1026

In [ ]:
aug_images = generate_aument_images(x_train[0:4500])

In [ ]:
x_train1 = x_train[0:4500]

In [ ]:
del x_train

In [ ]:
x_train = np.concatenate((x_train1,aug_images),axis=0)

In [ ]:
y_train1 = y_train[0:4500]

In [ ]:
y_aug = y_train[0:4500]

In [ ]:
del y_train

In [ ]:
y_train = np.concatenate((y_train1,y_aug), axis=0)

In [ ]:
del aug_images, y_aug, y_train1, x_train1

In [ ]:
gc.collect()

748

In [ ]:
train_model(x_train,y_train,x_val,y_val,saree_attr7,epochs=10)